# Introduction

<center><img src="https://cdn.mos.cms.futurecdn.net/RNrVwVfRiyoKkrr8djHvf9-650-80.jpg.webp"></center>
<center><font color="BBBBBB" size=2>(Image credit: Adobe Firefly - AI generated for Future, from Tom's guide)</font></center>

## Objective

Use Llama3 Langchain and ChromaDB to create a Retrieval Augmented Generation (RAG) system. This will allow us to ask questions about our documents (that were not included in the training data), without fine-tunning the Large Language Model (LLM).
When using RAG, if you are given a question, you first do a retrieval step to fetch any relevant documents from a special database, a vector database where these documents were indexed. 
The data that we will use is the text of EU AI Act, approved on March 13, 2024.

## Definitions

* LLM - Large Language Model  
* Llama3- LLM from Meta 
* Langchain - a framework designed to simplify the creation of applications using LLMs
* Vector database - a database that organizes data through high-dimmensional vectors  
* ChromaDB - vector database  
* RAG - Retrieval Augmented Generation (see below more details about RAGs)

## Model details

* **Model**: Llama 3  
* **Variation**: 8b-chat-hf  (8b: 8B dimm.; hf: HuggingFace)
* **Version**: V1  
* **Framework**: Transformers  

Llama3 model is pretrained and fine-tuned with 15T+ (more than 15 Trillion) tokens and 8 to 70 Billion parameters which makes it one of the powerful open source models. It is a highly improvement over Llama2 model.


## What is a Retrieval Augmented Generation (RAG) system?

Large Language Models (LLMs) has proven their ability to understand context and provide accurate answers to various NLP tasks, including summarization, Q&A, when prompted. While being able to provide very good answers to questions about information that they were trained with, they tend to hallucinate when the topic is about information that they do "not know", i.e. was not included in their training data. Retrieval Augmented Generation combines external resources with LLMs. The main two components of a RAG are therefore a retriever and a generator.  
 
The retriever part can be described as a system that is able to encode our data so that can be easily retrieved the relevant parts of it upon queriying it. The encoding is done using text embeddings, i.e. a model trained to create a vector representation of the information. The best option for implementing a retriever is a vector database. As vector database, there are multiple options, both open source or commercial products. Few examples are ChromaDB, Mevius, FAISS, Pinecone, Weaviate. Our option in this Notebook will be a local instance of ChromaDB (persistent).

For the generator part, the obvious option is a LLM. In this Notebook we will use a quantized Llama3 model, from the Kaggle Models collection.  

The orchestration of the retriever and generator will be done using Langchain. A specialized function from Langchain allows us to create the receiver-generator in one line of code.

## The data 

The data that will be indexed in the vector database to make it searchable by the RAG system is the complete text of the European Union Artificial Intelligence Act. This is a European Union regulation on Artificial Intelligence (AI) in the European Union. Proposed by the European Commission on 21 April 2021, it was adopted on 13 March 2024. 



# Installations, imports, utils

In [ ]:
!pip install transformers==4.33.0 accelerate==0.22.0 einops==0.6.1 langchain==0.0.300 xformers==0.0.21 \
bitsandbytes==0.41.1 sentence_transformers==2.2.2 chromadb==0.4.12

In [ ]:
import sys
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
#import chromadb
#from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma

# Initialize model, tokenizer, query pipeline

Define the model, the device, and the `bitsandbytes` configuration.

In [ ]:
model_id = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

print(device)

Prepare the model and the tokenizer.

In [ ]:
time_start = time()
model_config = transformers.AutoConfig.from_pretrained(
   model_id,
    trust_remote_code=True,
    max_new_tokens=1024
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
time_end = time()
print(f"Prepare model, tokenizer: {round(time_end-time_start, 3)} sec.")

Netx, we define the query pipeline.  
In order to work correctly when we will define the HuggingFace pipeline, we will need to define here the max_length (to avoid falling back on the very short default length of `20`.

In [ ]:
time_start = time()
query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        max_length=1024,
        device_map="auto",)
time_end = time()
print(f"Prepare pipeline: {round(time_end-time_start, 3)} sec.")

We define a function for testing the pipeline.

In [ ]:
def test_model(tokenizer, pipeline, message):
    """
    Perform a query
    print the result
    Args:
        tokenizer: the tokenizer
        pipeline: the pipeline
        message: the prompt
    Returns
        None
    """    
    time_start = time()
    sequences = pipeline(
        message,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=200,)
    time_end = time()
    total_time = f"{round(time_end-time_start, 3)} sec."
    
    question = sequences[0]['generated_text'][:len(message)]
    answer = sequences[0]['generated_text'][len(message):]
    
    return f"Question: {question}\nAnswer: {answer}\nTotal time: {total_time}"


## Test the query pipeline

We test the pipeline with few queries about European Union Artificial Intelligence Act (EU AI Act).

We also define here an utility function. This function will be used to display the output from the answer of the LLM.  
We include the calculation time, the question and the answer, formated so that will be easy to recognise them.

In [ ]:
from IPython.display import display, Markdown
def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

Let's test now the pipeline with few queries.

In [ ]:
response = test_model(tokenizer,
                    query_pipeline,
                   "Please explain what is EU AI Act.")
display(Markdown(colorize_text(response)))

In [ ]:
response = test_model(tokenizer,
                    query_pipeline,
                   "In the context of EU AI Act, how is performed the testing of high-risk AI systems in real world conditions?")
display(Markdown(colorize_text(response)))

The answer is not really useful. Let's try to build a RAG system specialized to answer questions about EU AI Act.

# Retrieval Augmented Generation

In order to build the RAG system, we will perform the following steps:
* Test the model using a HuggingFacePipeline;  
* Ingest the document using PyPdfLoader;
* Chunk the documents (with chunk size 1000), making sure we have also a partial overlap (of 100 characters);  
* Create embeddings and ingest the transformed text (text from pdf, chunked with overlap, embedded, and indexed) in the vector database;  
* Create the RequestQA pipeline (that includes the retrieval step and the generation step).

## Check the model with a HuggingFace pipeline


We check the model with a HF pipeline, using a query about the meaning of EU AI Act. We will need to use the HuggingFacePipeline in order to integrate easier with the Langchain tasks.

In [ ]:
llm = HuggingFacePipeline(pipeline=query_pipeline)

# checking again that everything is working fine
time_start = time()
question = "Please explain what EU AI Act is."
response = llm(prompt=question)
time_end = time()
total_time = f"{round(time_end-time_start, 3)} sec."
full_response =  f"Question: {question}\nAnswer: {response}\nTotal time: {total_time}"
display(Markdown(colorize_text(full_response)))

## Ingestion of data using PyPDFLoader

We will ingest the EU AI Act data using the PyPDFLoader from Langchain. There are actually multiple PDF ingestion utilities, we selected this one since it is easy to use.

In [ ]:
loader = PyPDFLoader("/kaggle/input/eu-ai-act-complete-text/aiact_final_draft.pdf")
documents = loader.load()

## Split data in chunks

We split data in chunks using a recursive character text splitter.  

Note: You can experiment with several values of chunk_size and chunk_overlap. Here we will set the following values:
* chunk_size: 1000 (this gives the size of a chunk, in characters). 
* chunk_overlap: 100 (this gives the number of characters with which two succesive chunks overlap).  

Chunk overlapping is required in order to be able to keep the context, even if we have a concept that we want to include that is spread over multiple document chunks.


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
all_splits = text_splitter.split_documents(documents)

## Creating Embeddings and Storing in Vector Store

Create the embeddings using Sentence Transformer and HuggingFace embeddings.  
Ocasionally, HuggingFace sentence-transformers might not be available. We implement therefore a mechanism to work with local stored sentence transformers.

In [ ]:

model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}

# try to access the sentence transformers from HuggingFace: https://huggingface.co/api/models/sentence-transformers/all-mpnet-base-v2
try:
    embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)
except Exception as ex:
    print("Exception: ", ex)
    # alternatively, we will access the embeddings models locally
    local_model_path = "/kaggle/input/sentence-transformers/minilm-l6-v2/all-MiniLM-L6-v2"
    print(f"Use alternative (local) model: {local_model_path}\n")
    embeddings = HuggingFaceEmbeddings(model_name=local_model_path, model_kwargs=model_kwargs)

Initialize ChromaDB with the document splits, the embeddings defined previously and with the option to persist it locally.  
We make sure to use the persistence option for the vector database.

In [ ]:
vectordb = Chroma.from_documents(documents=all_splits, embedding=embeddings, persist_directory="chroma_db")

## Initialize chain   

We are using `RetrievalQA` task chain utility from Langchain.  
This will first query the vector database (using similarity search) with the prompt we are using.   
Then, the query and the context retrieved (the documents that match with the query) are used to compose a prompt that instructs the LLM to answer to the query (**Generation**) using the information from the context retrieved (**Retrieval**). Therefore the name of the system, `Retrieval Augmented Generation`. 


In [ ]:
retriever = vectordb.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

## Test the Retrieval-Augmented Generation 


We define a test function, that will run the query and time it.

In [ ]:
def test_rag(qa, query):
    """
    Test the Retrieval Augmented Generation (RAG) system.
    
    Args:
        qa (RetrievalQA.from_chain_type): Langchain function to perform RAG
        query (str): query for the RAG system
    Returns:
        None
    """

    time_start = time()
    response = qa.run(query)
    time_end = time()
    total_time = f"{round(time_end-time_start, 3)} sec."

    full_response =  f"Question: {query}\nAnswer: {response}\nTotal time: {total_time}"
    display(Markdown(colorize_text(full_response)))

Let's check few queries.

In [ ]:
query = "How is performed the testing of high-risk AI systems in real world conditions?"
test_rag(qa, query)

In [ ]:
query = "What are the operational obligations of notified bodies?"
test_rag(qa, query)

In [ ]:
query = "What are the unacceptable risks?"
test_rag(qa, query)

In [ ]:
query = "In what cases a company that develops AI solutions should obtain permission to deploy it?"
test_rag(qa, query)

## Document sources

Let's check the documents sources, for the last query run.  

In order to do this, we will perform the following steps:
* We run a similarity search in the vector database;
* We loop through the documents returned; 
* Print, for each document, the documents source, from the metadata, and the page content.


In [ ]:
docs = vectordb.similarity_search(query)
print(f"Query: {query}")
print(f"Retrieved documents: {len(docs)}")
for doc in docs:
    doc_details = doc.to_json()['kwargs']
    print("Source: ", doc_details['metadata']['source'])
    print("Text: ", doc_details['page_content'], "\n")

# Conclusions


We used Langchain, ChromaDB and Llama3 as a LLM to build a Retrieval Augmented Generation solution. For testing, we were using the EU AI Act from 2023.  
The answers to questions from EU AI Act are correct, when using a RAG model.  

To improve the solution, we will have to refine the RAG implementation, first by optimizing the embeddings, then by using more complex RAG schemes.



